In [11]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

import requests
import gmaps
import os
from pprint import pprint

#import api key
from api_keys import g_key

In [22]:
#create dataframe from csv of breweries, courtesy of OpenBreweryDB (https://github.com/openbrewerydb/openbrewerydb)
brews_df = pd.read_csv ('openbrewerydb.csv')

#isolate breweries in the state of georgia
ga_brews_df = brews_df.loc [brews_df['state'] == 'Georgia', :]

#can exclude breweries in planning, or we can analyze them too
# ga_brews_df = ga_brews_df.loc [ga_brews_df['brewery_type'] != 'planning', :]

# ga_brews_df.head()
ga_brews_df

,id,name,brewery_type,street,city,state,postal_code,website_url,phone,created_at,updated_at,country,longitude,latitude,tags
2077,millercoors-brewing-co-albany,MillerCoors Brewing Co - Albany,large,405 Cordele Rd,Albany,Georgia,31705-2109,NaN,NaN,2018-07-24 01:33:15.988112,2018-08-24 00:29:13.337065,United States,-84.082031,31.585493,NaN
2078,pretoria-fields-collective,Pretoria Fields Collective,micro,120 Pine Ave,Albany,Georgia,31701-2529,http://www.pretoriafields.com,2.295182e+09,2018-07-24 01:33:16.20317,2018-08-24 00:29:21.824512,United States,-84.149593,31.578583,NaN
2079,currahee-brewing-company,Currahee Brewing Company,micro,25 S Main St,Alpharetta,Georgia,30009,NaN,NaN,2018-07-24 01:33:15.618382,2018-08-24 00:29:02.040773,United States,-84.296783,34.071110,NaN
2080,jekyll-brewing,Jekyll Brewing,micro,2855 Marconi Dr Ste 350,Alpharetta,Georgia,30005-2051,http://www.jekyllbrewing.com,7.705969e+09,2018-07-24 01:33:15.884802,2018-08-24 00:29:09.789612,United States,-84.253664,34.081656,NaN
2081,southern-brewing-company,Southern Brewing Company,micro,231 Collins Industrial Blvd,Athens,Georgia,30601-1517,http://www.sobrewco.com,7.065487e+09,2018-07-24 01:33:16.426119,2018-08-24 00:29:29.429091,United States,-83.350023,33.981140,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2176,brewery-in-planning-suwanee,Brewery in Planning - Suwanee,planning,NaN,Suwanee,Georgia,30024-4515,NaN,6.786898e+09,2018-07-24 01:33:15.493219,2018-08-11 21:37:01.183565,United States,NaN,NaN,NaN
2177,tucker-brewing-company,Tucker Brewing Company,micro,2003 S Bibb Dr,Tucker,Georgia,30084-6240,http://www.tuckerbrewing.com,8.337522e+09,2018-07-24 01:33:16.624197,2018-08-24 00:29:36.652788,United States,-84.207588,33.842692,NaN
2178,georgia-beer-co,Georgia Beer Co,planning,NaN,Valdosta,Georgia,31601-9017,http://www.georgiabeerco.com,9.123996e+09,2018-07-24 01:33:15.743498,2018-08-11 21:37:01.893856,United States,NaN,NaN,NaN
2179,chattabrewchee-southern-brewhouse,Chattabrewchee Southern Brewhouse,micro,709 4th Ave,West Point,Georgia,31833-1506,http://www.chattabrewchee.com,7.065011e+09,2018-07-24 01:33:15.554137,2018-08-24 00:28:59.773004,United States,-85.185362,32.876258,NaN


In [13]:
#analysis 1: comparing number of types of breweries

#gather frequency of types of breweries, display in dataframe
type_counts = ga_brews_df['brewery_type'].value_counts()

type_counts_df = pd.DataFrame (type_counts)
type_counts_df

,brewery_type
micro,46
brewpub,25
planning,24
large,3
regional,3
contract,2
proprietor,1


In [14]:
#using the type counts ^, we can compare numbers w/ a bar graph

#eduardo

In [15]:
#analysis 2: cities/areas w/ the most microbreweries/brewpubs

#use locate fxn to isolate small production breweries in georgia
ga_microbrews_df = ga_brews_df.loc [(ga_brews_df['brewery_type'] == 'micro')|
                                    (ga_brews_df['brewery_type'] == 'brewpub'), :]

#gather frequencies of these breweries per city
city_counts = ga_microbrews_df['city'].value_counts()
city_counts_df = pd.DataFrame (city_counts)

#pretty pie chart

#eduardo

In [16]:
#stacked bar chart using city & type counts in previous 2 cells
#eduardo

In [77]:
#shahil


#use lats/lngs to create heatmap on gmaps

#configure gmaps
gmaps.configure (api_key = g_key)

ga_brews_NA_df = ga_brews_df.dropna(subset=["longitude"])
ga_brews_NA_df = ga_brews_df.dropna(subset=["latitude"])

ga_brews_NA_df 


locs = ga_brews_NA_df[["latitude", "longitude"]].astype(float)

#display appropriate zoom level, center in atlanta (ish)
fig1 = gmaps.figure(zoom_level = 6, center = (34, -84))

#insert heatmap layer fxns here:
heat_layer = gmaps.heatmap_layer(locs)

fig1.add_layer(heat_layer)
#display figure
fig1


Figure(layout=FigureLayout(height='420px'))

In [75]:
#fig2 for atlanta specifically maybe

fig2 = gmaps.figure(zoom_level = 9, center = (33.8, -84.4))

fig2.add_layer(heat_layer)

fig2
#shahil

Figure(layout=FigureLayout(height='420px'))

In [1]:
#maria

#analysis 3: relation b/w breweries & colleges/univerisities

#gather list of colleges in georgia from google places; supported 'type' in google places is 'university'
#append lats/lngs to lists, populate dataframe
#create marker layer, add to figure

In [ ]:
#calculate distance b/w each microbrewery/pub & nearest university, add info to dataframe

#maria

In [ ]:
#distance bar chart thing
#maria

In [ ]:
#population demographics data, 1-2 images
#audrey

In [ ]:
#alternative ideas:
#establish date
#possible statistical test